In [ ]:
!pip install --upgrade transformers torch

In [ ]:
!pip install --upgrade bitsandbytes accelerate peft

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import set_seed


In [ ]:
dataset='dblp'
nome_llm='llama3' # flan # bloomz # llama13b

In [ ]:
# reprodutibilidade da solucao
import random, torch, numpy
SEED=42
random.seed(SEED); torch.manual_seed(SEED); numpy.random.seed(seed=SEED) # reproducibily soluction  


In [ ]:
if nome_llm == "llama3":
    nome_modelo = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(nome_modelo, 
            token=TOKEN_USUARIO, torch_dtype="auto", device_map="auto", load_in_4bit=True
    )
    tokenizer = AutoTokenizer.from_pretrained(nome_modelo, token=TOKEN_USUARIO)

elif nome_llm == 'flan':
    model_name='google/flan-t5-xxl' # "google/flan-t5-xl"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

def zero_shot(df, reference):
    results = []
    for i in range(len(df)):         
        if i%10==0:
            print(i)

        prompt = f"""Classify the topic of the text exclusively among the references:
Input: I bought new equipment to build my first robot.
Reference:
a. computer vision
b. computational linguistics
c. biomedical engineering
d. software engineering
e. graphics
f. data mining
g. security and cryptography
h. signal processing
i. robotics
j. theory
Answer: i
Input: {df.iloc[i]['text']}
Reference:
a. computer vision
b. computational linguistics
c. biomedical engineering
d. software engineering
e. graphics
f. data mining
g. security and cryptography
h. signal processing
i. robotics
j. theory
Answer: """

        if nome_llm == 'llama' or nome_llm == 'llama13b':
            inputs = tokenizer.encode(prompt, max_length=2048, return_tensors="pt").to("cuda")              
            #inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")    
        elif nome_llm == 'flan':
            inputs = tokenizer.encode(prompt, max_length=512, return_tensors="pt").to("cuda")   
        else:            
            inputs = tokenizer.encode(prompt,  max_length=1024,  return_tensors="pt").to("cuda") 
        
        set_seed(42)
        outputs = model.generate(inputs, max_new_tokens=1)
        
        results.append(tokenizer.decode(outputs[0][len(inputs[0]):]))
        
    df[nome_modelo] = results

In [ ]:
import timeit  # calcular metrica de tempo
#import torch
#with torch.autocast("cuda"): 

ini = timeit.default_timer()
df = pd.read_parquet(f"/kaggle/input/datasets-sentiment/{dataset}.parquet")

import string
letters = string.ascii_lowercase
# reference = "Reference:\n"
# name_classe = df['label'].unique()
# for i_classe in range(len(name_classe)):
#     reference += f"{letters[i_classe]}. {name_classe[i_classe]}\n"
reference=None  
#df = df.iloc[0:20]
for i in range(100):    
    print(f'parte:{i}')
    df_temp = df.iloc[i*1000:(i+1)*1000]
    zero_shot(df_temp, reference)
    df_temp.to_parquet(f"df_{dataset}_{nome_llm}_parte{i}.parquet")    
    if (i+1)*1000 > len(df):
        break
print(timeit.default_timer() - ini)
tempo_predicao = timeit.default_timer() - ini
#df

In [ ]:
import os
def list_files(dir):
    """Return list of the files in directory. Example: list_files('Downloads'); Return=['a.txt', 'b.txt']"""
    files_name =[]
    for r, d, files_array in os.walk(dir):
        for f in files_array:
            if '.parquet' in f:
                files_name.append(f)
    return files_name

In [ ]:
df = pd.DataFrame()
for i in range( len(list_files("./")) ):    
    df = pd.concat([df, 
               pd.read_parquet(f'./df_{dataset}_{nome_llm}_parte{i}.parquet')
              ])
df

In [ ]:
pred = []
for i in range(len(df)):
    if 'a' in df.iloc[i][nome_modelo]:
        pred.append(0)
    elif 'b' in df.iloc[i][nome_modelo]:
        pred.append(1)
    elif 'c' in df.iloc[i][nome_modelo]:
        pred.append(2)    
    elif 'c' in df.iloc[i][nome_modelo]:
        pred.append(3)  
    elif 'e' in df.iloc[i][nome_modelo]:
        pred.append(4)  
    elif 'f' in df.iloc[i][nome_modelo]:
        pred.append(5)  
    elif 'g' in df.iloc[i][nome_modelo]:
        pred.append(6)  
    elif 'h' in df.iloc[i][nome_modelo]:
        pred.append(7)  
    elif 'i' in df.iloc[i][nome_modelo]:
        pred.append(8)
    else:
        pred.append(9) #theory , majoriataria        
df[nome_modelo +'_pred'] = pred
 

In [ ]:
import collections

collections.Counter(df['meta-llama/Meta-Llama-3.1-8B-Instruct'])

In [ ]:
from sklearn.metrics import f1_score

for index_fold in range(5):
    folds = pd.read_parquet(f'/kaggle/input/datasets-sentiment/{dataset}_folds.parquet')
    test = df.iloc[folds.iloc[index_fold]['test_idxs']]
    micro = f1_score(list(test['label']), list(test[nome_modelo +'_pred']), average='micro')
    macro = f1_score(list(test['label']), list(test[nome_modelo +'_pred']), average='macro')
    f1_classes = f1_score(list(test['label']), list(test[nome_modelo +'_pred']), average=None)

    print(f"{dataset}_{nome_modelo}\t{index_fold}\t{micro}\t{macro}\t{tempo_predicao}\t{list(test[nome_modelo +'_pred'])}")
    #print(f"{dataset}_{nome_modelo}\t{index_fold}\t{micro}\t{macro}\t\t{f1_classes[0]}\t{f1_classes[1]}")